In [ ]:
import importlib
import pandas
import numpy
import re

In [ ]:
# from functions.get_mon_data import *
from functions.functions_dataframe import * 
# from functions.functions_shared import *
from parameters.filters_lists import *
from parameters.route import *
from parameters.regs import *

In [ ]:
"""
Завантажити бази шкіл і садків МОН з data.gov.ua
"""
expdata_sc = pandas.read_csv('data/mon_data/expdata_sc.csv', sep = ';')
expdata_dnz = pandas.read_csv('data/mon_data/expdata_dnz.csv', sep = ';')

In [ ]:
"""
Фільтрування бази МОН шкільних закладів за довжиною ЄДРПОУ коду (залишити тільки 7-8 значні коди, 
інші вважаються помилковими)
"""
expdata_sc = expdata_sc.fillna(0)
expdata_sc['edrpou'] = expdata_sc['edrpou'].astype(np.int64)
mask = (expdata_sc['edrpou'].astype(str).str.len() == 7) | (expdata_sc['edrpou'].astype(str).str.len() == 8)
expdata_sc = expdata_sc.loc[mask].sort_values(by=['edrpou'])

In [ ]:
"""
Фільтрування бази МОН дитячих садків за довжиною ЄДРПОУ коду (залишити тільки 7-8 значні коди, 
інші вважаються помилковими)
"""
expdata_dnz['edrpou'] = pd.to_numeric(expdata_dnz['edrpou'],errors='coerce')
expdata_dnz = expdata_dnz.fillna(0)
expdata_dnz['edrpou'] = expdata_dnz['edrpou'].astype(np.int64)
mask = (expdata_dnz['edrpou'].astype(str).str.len() == 7) | (expdata_dnz['edrpou'].astype(str).str.len() == 8)
expdata_dnz = expdata_dnz.loc[mask].sort_values(by=['edrpou'])

In [ ]:
"""
Об'єднати датафрейми садків і шкіл в один датафрейм
"""
frames = [expdata_dnz, expdata_sc]
df_mon_data = pd.concat(frames)

In [ ]:
"""
З бази ProZorro df_raw створений перелік ЄДРПОУ управлінь освіти, які трапляються в списку МОН
"""
df_prozorro = pandas.read_csv('data/TrainingProZorroForSchoolProject20162019.csv', sep = ';', low_memory = False)
filter_keywords_exclude = ["департамент", "управлін.*освіт", "відділ.*освіт"]
df_raw_upravlin_osvity = df_prozorro[df_prozorro['Организатор'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords_exclude)), case = False, na = False, regex = True)]
edrpou_upravlin_osvity_prozorro = df_raw_upravlin_osvity['IDOrganizator'].unique().tolist()

In [ ]:
"""
Виключити з переліку бази МОН df_mon_data освітні департаменти, управління освіти
"""
mask = (df_mon_data['edrpou'].isin(edrpou_upravlin_osvity_prozorro))
df_mon_data_bez_upravlin_osvity = df_mon_data.loc[~mask]

In [ ]:
"""
Зберегти датафрейм МОН навчальних закладів, з якого виделані департаменти, управління, відділи
"""
df_mon_data_bez_upravlin_osvity.to_csv('data/MON_schools_dnz.csv', sep=';')

### Створити список власних назв шкіл і садків

In [ ]:
"""
Залишити з датафрейму df_mon_data_bez_upravlin_osvity одну колонку - з назвами шкіл і садків 
"""
df_university_names = df_mon_data_bez_upravlin_osvity.loc[:, df_mon_data_bez_upravlin_osvity.columns.intersection(['university_name'])]

In [ ]:
"""
Замінити лапки типу «» на " 
"""
df_university_names.university_name = df_university_names.university_name.str.replace('«|»', '"')

In [ ]:
"""
Створити колонку "Назва" зі знайденими у лапках словами
"""
df_university_names_filtered = naming_one_column(df_university_names, ['university_name'], reg_double_quotes)

In [ ]:
"""
Видалити пусті рядки
"""
df_university_names_filtered.dropna(inplace=True)

In [ ]:
searchfor = ['№', 'заклад']
df_names_clean = df_university_names_filtered[~df_university_names_filtered["Назва"].str.contains('|'.join(searchfor), case = False)]

In [ ]:
"""
Видалити пусті рядки
"""
df_names_clean.dropna(inplace=True)

In [ ]:
"""
Видалити символи "()
"""
df_names_clean['Назва'] = df_names_clean['Назва'].str.replace('"', '')
df_names_clean['Назва'] = df_names_clean['Назва'].str.replace(')', '')
df_names_clean['Назва'] = df_names_clean['Назва'].str.replace('(', '')

In [ ]:
"""
Зберегти в окремий файл df_name_list.csv
"""
df_names_clean.to_csv('data/MON_names.csv', sep=';')